In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
URL = "https://www.indeed.co.uk/jobs?as_and=music&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=&radius=25&l=United+Kingdom&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch"

page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/23a063b/en_GB.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/b45d10b/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="https://www.indeed.co.uk/rss?q=music&amp;l=United+Kingdom" rel="alternate" title="Music Jobs, vacancies in United Kingdom" type="application/rss+xml"/>
  <link href="/m/jobs?q=music&amp;l=United+Kingdom&amp;limit=50" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=music&amp;l=United+Kingdom&amp;limit=50" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
  </sc

In [25]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

jobs = extract_job_title_from_result(soup)

#source
#https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

In [26]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
companies = extract_company_from_result(soup)

#source
#https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

In [53]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll(["div", "span"], attrs={"class": "location accessible-contrast-color-location"})
    for span in spans:
        if len(span) > 0:
            locations.append(span.text)
        else:
            locations.append("Not Provided")
    return(locations)

locations = extract_location_from_result(soup)

#source
#https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

In [40]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("div", attrs={"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

summaries = extract_summary_from_result(soup)

#source
#https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

In [55]:
indeed_df = pd.DataFrame(np.random.randint(0,100,size=(50,4)), columns=['jobTitle', 'jobDescription', 'location', 'companyName'])

In [54]:
print(len(jobs))
print(len(locations))
print(len(summaries))
print(len(companies))

50
50
50
50


In [56]:
indeed_df['jobTitle'] = np.resize(jobs,len(indeed_df))

In [58]:
indeed_df['jobDescription'] = np.resize(summaries,len(indeed_df))

In [59]:
indeed_df['location'] = np.resize(locations,len(indeed_df))

In [60]:
indeed_df['companyName'] = np.resize(companies,len(indeed_df))

In [61]:
indeed_df.head()

jobTitle  \
0       Freelance Music Journalists and Broadcasters   
1  Beautiful Digital - Digital Account Manager fo...   
2  Music Sales & Business Development Representative   
3                                  Guitar Technician   
4           Unqualified Primary Teacher - ASAP Start   

                                      jobDescription  \
0  Part-time hours: 15 per week.\nJob Types: Part...   
1  This is a 12 month + working from home Apprent...   
2  They often pitch directly to Radio, TV, Press ...   
3  The role may also include (depending on time a...   
4  The Head Teacher is looking to invest into an ...   

                   location             companyName  
0                 Cambridge    www.quitegreat.co.uk  
1            London E17 6DS  BIG CREATIVE EDUCATION  
2                   Reigate           Music Gateway  
3  Stratford-upon-Avon CV37        Richards Guitars  
4                    Newham        Clarus Education

In [62]:
indeed_df.shape

(50, 4)

In [65]:
indeed_df.to_csv('indeed.csv', index = False, header=True, encoding='utf-8')

Source for all notebook
https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b
as of 16/10/20